In [1]:
import pandas as pd
import googletrans
from googletrans import Translator
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

## Importing the file

In [2]:
df=pd.read_csv('music_collection.csv')

#Checking for nulls
df.isnull().sum()

cols_drop=['Rating','Collection Media Condition','Collection Sleeve Condition','Collection Notes']
df=df.drop(cols_drop,axis=1)

,Catalog#,Artist,Title,Label,Format,Released,release_id,CollectionFolder,Date Added,date,Time
0,"PIASV 007 CD, 946.0007.128",Mogwai,My Father My King,"[PIAS] Recordings, [PIAS] Recordings, Rock Act...","CD, EP, Single",2001,527808,Uncategorized,07/12/2019 22:43,2019-07-12,22:43:00
1,12t030,Lost In Hildurness,Mount A,12 Tónar,"CD, Album",2006,948733,Uncategorized,07/12/2019 22:11,2019-07-12,22:11:00
2,12K1032,Taylor Deupree + Kenneth Kirschner,Post_Piano 2,12k,"CD, Album, Ltd",2005,464491,Uncategorized,20/12/2019 10:45,2019-12-20,10:45:00
3,12K1035,Sawako,Hum,12k,"CD, Album, Ltd",2005,563530,Uncategorized,25/11/2019 00:07,2019-11-25,00:07:00
4,12K1040,Seaworthy,Map In Hand,12k,"CD, Album, RE, RM",2006,828844,Uncategorized,25/11/2019 00:05,2019-11-25,00:05:00
...,...,...,...,...,...,...,...,...,...,...,...
694,"XLLP868, XLLP868X",Radiohead,OK Computer OKNOTOK 1997 2017,"XL Recordings, XL Recordings","3xLP, Album, Ltd, RE, RM, Blu",2017,10544976,Uncategorized,02/11/2019 11:08,2019-02-11,11:08:00
695,GR-021,Raum (4),Daughter,Yellow Electric,"LP, Ltd + LP, S/Sided + Album",2022,22631921,Uncategorized,13/11/2022 04:34,2022-11-13,04:34:00
696,yeb-7746,Jun Miyake,Lost Memory Theatre - Act-2,Yellowbird,"CD, Album",2014,6969394,Uncategorized,07/12/2019 22:56,2019-07-12,22:56:00
697,MIR 100708,Эдуард Артемьев,Solaris - Music From The Motion Picture By And...,Мирумир,"LP, Album",2013,4837043,Uncategorized,03/11/2019 10:53,2019-03-11,10:53:00


### Date and time functions

In [ ]:
def date_conv(df,date):
    df['date']=pd.to_datetime(df['Date Added'])
    df['date']=df['date'].dt.date
    return df

date_conv(df,'date')

def time_conv(df,Time):
    df['Time']=pd.to_datetime(df['Date Added'])
    df['Time']=df['Time'].dt.time
    return df

time_conv(df,'Time')

### Translating and cleaning record labels and album titles

In [3]:
translator=Translator()

In [6]:
df_copy=df.copy()
df_columns=['Label','Title']

translations = {}
for column in df_columns:
    # unique elements of the column
    unique_elements = df_copy[column].unique()
    for element in unique_elements:
        # add translation to the dictionary
        translations[element] = translator.translate(element).text
    
print(translations)

{'[PIAS] Recordings, [PIAS] Recordings, Rock Action Records, Rock Action Records': '[PIAS] Recordings, [PIAS] Recordings, Rock Action Records, Rock Action Records', '12 Tónar': '12 Tones', '12k': '12k', '130701': '130701', '1703 Skivbolaget': '1703 The record company', '20|20|20': '20|20|20', '2nd Rec': '2nd Rec', '4AD': '4AD', '4AD, 4AD': '4AD, 4AD', '52Hz': '52Hz', '52Hz, 52Hz': '52Hz, 52Hz', '8mm Records': '8mm Records', 'A&M Records, UMe': 'A&M Records, UMe', 'Aagoo Records, REV. Laboratories': 'Aagoo Records, REV. Laboratories', 'Aerophonic Records': 'Aerophonic Records', 'Alegra': 'Alegra', 'Alien8 Recordings': 'Alien8 Recordings', 'All Is Number Records': 'All Is Number Records', 'Another Timbre': 'Another Timbre', 'Anti-': 'Anti-', 'Anticipate Recordings': 'Anticipate Recordings', 'Anticon, Alien Transistor': 'Anticon, Alien Transistor', 'Antifrost': 'Antifrost', 'Arbor': 'Arbor', 'Argeïphontes': 'Argeïphontes', 'Ars Benevola Mater': 'The Art of the Benevolent Mother', 'Asphode

In [9]:
def split_new_name(df,Label):
    new=df['Label'].str.split(",",n=1,expand=True)
    df['Label1']=new[0]
    df['Label2']=new[1]
    df['Label1']=df['Label1'].str.strip()
    df['Label2']=df['Label2'].str.strip()
    return df

split_new_name(df_copy,'Label')


'\nnew=df_copy[\'Label\'].str.split(",",n=1,expand=True)\ndf_copy[\'Label1\']=new[0]\ndf_copy[\'Label2\']=new[1]\n'

In [11]:
df_copy['Same_label']=np.where(df_copy['Label1']==df_copy['Label2'],'Same','Different')
df_copy['Format']=df_copy['Format'].astype(str)

df_copy['Format']=df_copy['Format'].str.replace('\d+','') #removes the digit from the format 
df_copy['Format_category']=np.where(df_copy['Format'].str.contains("CD"),"CD",
                                    np.where(df_copy['Format'].str.contains("LP"),"LP",df_copy['Format']))

# Generating random values for 
df_copy['Value']=np.where(df_copy['Format'].str.contains('LP'),np.random.randint(10,50,df_copy.shape[0]),
                                                                                 np.random.randint(2,8,df_copy.shape[0]))

C:\Users\KKI27\AppData\Local\Temp\ipykernel_21196\4054993276.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_copy['Format']=df_copy['Format'].str.replace('\d+','') #removes the digit from the format


## File export

In [90]:
df_top_artists=pd.concat([df_hecker,df_ambarchi,df_davachi,df_coil,df_grouper],axis=0)

In [94]:
df_top_artists.columns

Index(['Catalog#', 'Artist', 'Title', 'Label', 'Format', 'Released',
       'release_id', 'CollectionFolder', 'Date Added', 'date', 'Time',
       'Label1', 'Label2', 'Same_label', 'Format_category', 'Value',
       'Date Added Year'],
      dtype='object')

In [12]:
df_copy.to_csv('music_collection_final.csv')